In [1]:
import os
# import httpx

import mlflow

from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIChatModel

## Setup mlflow logging
# Turn on auto tracing by calling mlflow.pydantic_ai.autolog()
mlflow.pydantic_ai.autolog()

# Optional: Set a tracking URI and an experiment
mlflow.set_tracking_uri("http://localhost:9041")
mlflow.set_experiment("Default")

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1771577890363, experiment_id='0', last_update_time=1771577890363, lifecycle_stage='active', name='Default', tags={}>

In [ ]:
# Setup Model
MODEL="Qwen3-VL-30B-A3B-Instruct"

# Passing AsyncOpenAI currently causes following error
# 'ASyncHttpxClientWrapper' object has no attribute '_state'
# llm_client=AsyncOpenAI(
#     base_url="http://localhost:9001/v1",
#     api_key="",
# )

provider = OpenAIProvider(
    # openai_client=llm_client,
    base_url="http://localhost:9001/v1",
    api_key="",
)
model = OpenAIChatModel(MODEL, provider=provider)

In [3]:
# Simple Example
class WeatherInfo(BaseModel):
    location: str
    temperature: float = Field(description="Temperature in Celsius")
    condition: str
    humidity: int = Field(description="Humidity percentage")

# Create an agent with system prompts and tools
weather_agent = Agent(
    model=model,
    output_type=WeatherInfo,
    system_prompt="You are a helpful weather assistant. Always provide accurate weather information.",
    instrument=True
)

@weather_agent.tool
async def get_weather_data(ctx: RunContext[None], location: str) -> str:
    """Get current weather data for a location."""
    # This is a placeholder - use a real weather API
    mock_data = {
        "temperature": 22.5,
        "condition": "partly cloudy",
        "humidity": 65
    }
    return f"Weather in {location}: {mock_data}"

# Run the agent with tool usage

In [4]:
result = await weather_agent.run("What's the weather like in Paris?")
print(result)

AgentRunResult(output=WeatherInfo(location='Paris', temperature=22.5, condition='partly cloudy', humidity=65))


Trace(trace_id=tr-eb0ee701c31485f14c02d836c5fa9d42)